In [1]:
def convert_back(event_type, event_result):
    if event_type == "Field":
        feet = int(event_result // 12)
        inches = event_result - (feet * 12)   
        return str(feet) + "'" + " " + '{:g}'.format(inches) + "\""
    else:
        minutes = int(event_result // 60)
        seconds = event_result - (minutes * 60)

        if minutes == 0:
            return "{:.2f}".format(seconds)
        else:
            if seconds < 10:
                return str(minutes) + ":0" + "{:.2f}".format(seconds)
            else:
                return str(minutes) + ":" + "{:.2f}".format(seconds)

In [116]:
import pandas as pd
from Documents.GitHub.trackinsights.db_util import Database

db = Database("C:\\Users\\rritz\\Track Results")

df = db.get_all_athlete_results()
df_relay = db.get_all_relay_results()

# list for genders
gender_list = ['Girls', 'Boys']

# list for events
events = []

# track events
events.append("100 Meters");
events.append("200 Meters");
events.append("400 Meters");
events.append("800 Meters");
events.append("1600 Meters");
events.append("3200 Meters");
events.append("110 Hurdles");
events.append("300 Hurdles");

# relay events
events.append("4 x 100 Relay");
events.append("4 x 400 Relay");
events.append("4 x 800 Relay");

# field events
events.append("High Jump");
events.append("Long Jump");
events.append("Discus");
events.append("Shot Put");
events.append("Pole Vault");

percentiles = [0.25, 0.50, .58, .75, .85, .95]

# create a new DataFrame to store results
resultDF = pd.DataFrame([], columns= percentiles)

for event_gender in gender_list:
    
    # loop over all events in the events list
    for event_name in events:
        
        if event_gender == "Girls" and event_name == "110 Hurdles":
            event_name = "100 Hurdles";

        if "Relay" in event_name:
            df2 = df_relay[ (df_relay.meet_type == "Sectional") & (df_relay.event == event_name) & (df_relay.gender == event_gender) & (df_relay.result2 != 0) & (df_relay.result2 != 9999)]           
        else:
            df2 = df[ (df.meet_type == "Sectional") & (df.event == event_name) & (df.gender == event_gender) & (df.result2 != 0) & (df.result2 != 9999)]           

        if event_name[0].isdigit():
            event_type = "Track"
            track_percentiles = [1 - x for x in percentiles]
            df3 = df2['result2'].quantile(track_percentiles) 
        else:
            event_type = "Field"
            df3 = df2['result2'].quantile(list(reversed(percentiles)))
 
        index = event_name

        percentile_values = []

        for i in range (len(percentiles)):
            percentile_values.append(convert_back(event_type, df3.iloc[i]))
            
        resultDF.loc[index] = percentile_values

            
    display(resultDF)
    resultDF.drop(resultDF.index, inplace=True)

,0.25,0.50,0.58,0.75,0.85,0.95
100 Meters,14.13,13.56,13.40,13.09,12.88,12.56
200 Meters,29.65,28.24,27.93,27.28,26.83,26.11
400 Meters,1:10.97,1:06.88,1:05.72,1:03.45,1:01.63,59.69
800 Meters,2:55.88,2:43.52,2:39.85,2:32.48,2:26.89,2:20.70
1600 Meters,6:34.39,6:05.10,5:57.10,5:40.22,5:28.75,5:13.95
3200 Meters,14:07.93,13:07.56,12:52.18,12:15.22,11:50.83,11:18.86
100 Hurdles,19.02,17.79,17.50,16.80,16.32,15.49
300 Hurdles,56.90,53.23,52.46,50.47,49.07,46.85
4 x 100 Relay,55.77,53.59,53.02,51.49,50.79,49.44
4 x 400 Relay,4:43.38,4:28.45,4:23.73,4:16.64,4:11.45,4:04.01


,0.25,0.50,0.58,0.75,0.85,0.95
100 Meters,12.18,11.76,11.65,11.43,11.28,11.07
200 Meters,24.96,24.00,23.80,23.33,23.02,22.48
400 Meters,57.73,55.11,54.40,52.85,51.87,50.30
800 Meters,2:22.15,2:12.26,2:09.67,2:04.75,2:01.82,1:58.50
1600 Meters,5:21.80,5:00.30,4:54.28,4:41.94,4:33.06,4:25.37
3200 Meters,11:40.71,10:47.02,10:37.02,10:11.10,9:55.75,9:37.13
110 Hurdles,18.34,16.91,16.62,15.99,15.65,15.04
300 Hurdles,47.58,44.66,44.03,42.77,41.69,40.29
4 x 100 Relay,47.30,45.54,45.08,44.10,43.40,42.56
4 x 400 Relay,3:49.65,3:39.81,3:37.17,3:32.10,3:29.25,3:24.07
